# LoRA fine-tuning for WMT14
本 Notebook 复现 `src/train_lora_mt.py` 的训练流程，并按照“环境与依赖准备 → 数据处理 → 模型搭建 → 训练 → 验证 → 推理与保存”的顺序组织，使其更适合交互式实验。

## 1. 环境与依赖准备
在此单元中安装/校验依赖、设置路径与随机种子，并准备硬件检测与公用工具函数。

In [ ]:
# 依据需要安装依赖（若环境已满足可跳过）
!pip install -q transformers peft datasets accelerate evaluate bitsandbytes

In [ ]:
import os
import math
import inspect
import logging
from pathlib import Path
from typing import Any, Dict, Iterable

import numpy as np
import torch
from datasets import Dataset, DatasetDict, load_dataset
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    TrainerCallback,
    TrainerState,
    TrainerControl,
    set_seed,
)
import yaml

# 允许 notebook 直接复用项目模块
PROJECT_ROOT = Path("..").resolve()
SRC_DIR = PROJECT_ROOT / "src"
import sys

if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from data_utils import build_preprocess_function, load_translation_dataset  # noqa: E402

os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)
LOGGER = logging.getLogger("train_lora_mt_notebook")


def load_config(config_path: str) -> Dict[str, Any]:
    with open(config_path, "r", encoding="utf-8") as handle:
        return yaml.safe_load(handle)


def get_dtype(dtype_name: str) -> torch.dtype:
    lowered = dtype_name.lower()
    if lowered in {"bfloat16", "bf16"}:
        return torch.bfloat16
    if lowered in {"float16", "fp16"}:
        return torch.float16
    return torch.float32


def detect_device_type() -> str:
    if torch.cuda.is_available():
        return "cuda"
    if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():  # type: ignore[attr-defined]
        return "mps"
    return "cpu"


def prepare_training_arguments(project_cfg: Dict[str, Any], training_cfg: Dict[str, Any], output_dir: Path) -> Dict[str, Any]:
    signature = inspect.signature(Seq2SeqTrainingArguments.__init__)
    valid_params = set(signature.parameters.keys())
    rename = {"evaluation_strategy": ("evaluation_strategy", "eval_strategy")}
    args_dict: Dict[str, Any] = {}
    for key, value in training_cfg.items():
        dest = key
        if key in rename:
            for candidate in rename[key]:
                if candidate in valid_params:
                    dest = candidate
                    break
            else:
                LOGGER.warning("Dropping unsupported training arg: %s", key)
                continue
        if dest in valid_params:
            args_dict[dest] = value
        else:
            LOGGER.warning("Dropping unsupported training arg: %s", dest)
    args_dict["output_dir"] = str(output_dir)
    if "logging_dir" in valid_params:
        args_dict["logging_dir"] = project_cfg.get("logging_dir", "runs")
    return args_dict


def enable_gradient_checkpointing(model):
    base_model = getattr(getattr(model, "base_model", None), "model", model)
    base_model.gradient_checkpointing_enable()
    if hasattr(base_model, "enable_input_require_grads"):
        base_model.enable_input_require_grads()
    else:
        input_embeddings = base_model.get_input_embeddings()
        if input_embeddings is not None:
            def _require_grad(_, __, output):
                if isinstance(output, torch.Tensor):
                    output.requires_grad_(True)

            input_embeddings.register_forward_hook(_require_grad)
    if hasattr(model, "config"):
        model.config.use_cache = False


def set_global_seed(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    set_seed(seed)


print("Environment ready. SRC_DIR=", SRC_DIR)

## 2. 加载与预处理数据集
这一部分读取 YAML 配置、通过 🤗 Datasets 拉取 WMT14 数据，构建清洗 + tokenize 流程，并用 PyTorch DataLoader 缓存分片以便多任务/多进程场景复用。

In [ ]:
CONFIG_PATH = PROJECT_ROOT / "configs" / "lora_mt.yaml"
OUTPUT_OVERRIDE = None  # 可选：手动覆盖输出目录

config = load_config(str(CONFIG_PATH))
if OUTPUT_OVERRIDE:
    config["project"]["output_dir"] = str(OUTPUT_OVERRIDE)

project_cfg = dict(config["project"])
data_cfg = dict(config["data"])
model_cfg = dict(config["model"])
lora_cfg = dict(config["lora"])
training_cfg = dict(config["training"])  # 后续可在 Notebook 内动态修改
training_cfg.setdefault("dataloader_pin_memory", True)

set_global_seed(project_cfg.get("seed", 42))
device_type = detect_device_type()
LOGGER.info("Detected backend: %s", device_type)

if model_cfg.get("use_4bit") and device_type != "cuda":
    LOGGER.warning("4-bit quantization requires CUDA; disabling.")
    model_cfg["use_4bit"] = False

def _split_query(base: str, max_key: str) -> str:
    max_samples = data_cfg.get(max_key)
    return f"{base}[:{int(max_samples)}]" if max_samples else base

train_query = _split_query("train", "max_train_samples")
eval_base = "validation"
eval_query = _split_query(eval_base, "max_eval_samples")

try:
    raw_datasets = load_translation_dataset(
        data_cfg["dataset_name"],
        data_cfg.get("dataset_config"),
        cache_dir=project_cfg.get("cache_dir"),
        split_overrides={"train": train_query, eval_base: eval_query},
    )
    eval_split = eval_base
except ValueError:
    eval_base = "test"
    eval_query = _split_query(eval_base, "max_eval_samples")
    raw_datasets = load_translation_dataset(
        data_cfg["dataset_name"],
        data_cfg.get("dataset_config"),
        cache_dir=project_cfg.get("cache_dir"),
        split_overrides={"train": train_query, eval_base: eval_query},
    )
    eval_split = eval_base

LOGGER.info("Loaded splits: train=%s, eval=%s", train_query, eval_query)

tokenizer = AutoTokenizer.from_pretrained(model_cfg["name"], cache_dir=project_cfg.get("cache_dir"))
tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token
tokenizer.src_lang = model_cfg["tokenizer_src_lang_code"]
tokenizer.tgt_lang = model_cfg["tokenizer_tgt_lang_code"]

preprocess_fn = build_preprocess_function(tokenizer, data_cfg)
remove_columns = raw_datasets["train"].column_names
processed_datasets = raw_datasets.map(
    preprocess_fn,
    batched=True,
    remove_columns=remove_columns,
    desc="Tokenizing dataset",
)

preview_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None)
train_loader = DataLoader(
    processed_datasets["train"],
    batch_size=training_cfg["per_device_train_batch_size"],
    shuffle=True,
    collate_fn=preview_collator,
)

first_batch = next(iter(train_loader))
print("Sample batch keys:", first_batch.keys())
print("input_ids shape:", first_batch["input_ids"].shape)
print("labels shape:", first_batch["labels"].shape)


## 3. 构建基础模型与 LoRA 适配器
加载 mBART 主干、可选 4-bit 量化，配置 LoRA Rank/Alpha/Dropout 并仅训练目标模块，最后检查可训练参数比例。

In [ ]:
dtype = get_dtype(model_cfg.get("torch_dtype", "float32"))
if device_type != "cuda" and dtype in (torch.float16, torch.bfloat16):
    LOGGER.warning("%s backend detected; falling back to float32 weights.", device_type)
    dtype = torch.float32

if device_type != "cuda":
    for flag in ("bf16", "fp16", "gradient_checkpointing"):
        if training_cfg.get(flag):
            LOGGER.info("Disabling %s for %s backend.", flag, device_type)
            training_cfg[flag] = False
    if training_cfg.get("dataloader_num_workers", 0) > 0:
        training_cfg["dataloader_num_workers"] = 0
    training_cfg["dataloader_pin_memory"] = False

model_kwargs: Dict[str, Any] = {"cache_dir": project_cfg.get("cache_dir")}
if model_cfg.get("use_4bit"):
    quant_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
    model_kwargs["quantization_config"] = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=quant_dtype,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )
    model_kwargs["device_map"] = "auto"
else:
    model_kwargs["torch_dtype"] = dtype

model = AutoModelForSeq2SeqLM.from_pretrained(model_cfg["name"], **model_kwargs)
if model_cfg.get("use_4bit"):
    model = prepare_model_for_kbit_training(model)

if hasattr(model.config, "forced_bos_token_id") and hasattr(tokenizer, "lang_code_to_id"):
    model.config.forced_bos_token_id = tokenizer.lang_code_to_id[model_cfg["tokenizer_tgt_lang_code"]]

task_type = TaskType.SEQ_2_SEQ_LM if lora_cfg["task_type"].lower() == "seq2seq_lm" else TaskType.SEQ_CLS
peft_config = LoraConfig(
    r=lora_cfg["r"],
    lora_alpha=lora_cfg["alpha"],
    target_modules=lora_cfg["target_modules"],
    lora_dropout=lora_cfg["dropout"],
    bias=lora_cfg.get("bias", "none"),
    task_type=task_type,
)
model = get_peft_model(model, peft_config)
if training_cfg.get("gradient_checkpointing"):
    enable_gradient_checkpointing(model)
model.print_trainable_parameters()

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable params: {trainable_params/1e6:.2f}M / {total_params/1e6:.2f}M ({100*trainable_params/total_params:.2f}%)")

## 4. 实现 `train_lora_mt` 训练循环
此处复现脚本训练逻辑：自定义多任务损失聚合 $\mathcal{L}_{total}=\sum_i w_i\mathcal{L}_i$，配置优化器/调度器、梯度累积与断点恢复。

In [ ]:
loss_weights = {"translation": 1.0}  # 可按任务维度调整


class WeightedSeq2SeqTrainer(Seq2SeqTrainer):
    """简单示例：按 task_id 聚合多任务损失，可扩展为真实多任务场景。"""

    def compute_loss(self, model, inputs, return_outputs=False):
        task_ids = inputs.pop("task_id", None)
        outputs = model(**inputs)
        loss = outputs.loss
        if task_ids is not None:
            # task_id -> 权重；本例只有 translation 任务
            unique_ids, counts = torch.unique(task_ids, return_counts=True)
            weighted_loss = torch.zeros_like(loss)
            for task, count in zip(unique_ids.tolist(), counts.tolist()):
                weight = loss_weights.get(task, 1.0)
                weighted_loss = weighted_loss + weight * loss / max(count, 1)
            loss = weighted_loss
        if return_outputs:
            return loss, outputs
        return loss


output_dir = Path(project_cfg["output_dir"])
output_dir.mkdir(parents=True, exist_ok=True)
training_args = Seq2SeqTrainingArguments(**prepare_training_arguments(project_cfg, training_cfg, output_dir))
LOGGER.info("Training arguments: %s", training_args)

trainer = WeightedSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_datasets["train"],
    eval_dataset=processed_datasets[eval_split],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

resume_checkpoint = project_cfg.get("resume_from_checkpoint")
train_result = trainer.train(resume_from_checkpoint=resume_checkpoint)
trainer.save_model()

train_metrics = train_result.metrics
train_metrics["train_samples"] = len(processed_datasets["train"])
trainer.log_metrics("train", train_metrics)
trainer.save_metrics("train", train_metrics)
trainer.save_state()

print("Training complete.")

## 5. 验证与指标可视化
实现 `evaluate()` 计算 BLEU/Perplexity/Accuracy，并把日志写入 pandas，借助 matplotlib 可视化 loss 与学习率。

In [ ]:
import evaluate
import pandas as pd
import matplotlib.pyplot as plt

bleu_metric = evaluate.load("sacrebleu")

def evaluate_model(max_length: int | None = None, num_beams: int | None = None) -> Dict[str, float]:
    eval_kwargs = {}
    if max_length:
        eval_kwargs["max_length"] = max_length
    if num_beams:
        eval_kwargs["num_beams"] = num_beams
    eval_raw = trainer.evaluate(**eval_kwargs)

    predicts = trainer.predict(processed_datasets[eval_split], **eval_kwargs)
    decoded_preds = tokenizer.batch_decode(predicts.predictions, skip_special_tokens=True)
    labels = np.where(predicts.label_ids != -100, predicts.label_ids, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    bleu = bleu_metric.compute(predictions=decoded_preds, references=[[lbl] for lbl in decoded_labels])
    exact_match = np.mean([int(p == l) for p, l in zip(decoded_preds, decoded_labels)])
    ppl = math.exp(min(eval_raw["eval_loss"], 20))

    metrics = {
        **eval_raw,
        "bleu": bleu["score"],
        "exact_match": float(exact_match),
        "perplexity": float(ppl),
    }
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)
    return metrics


eval_metrics = evaluate_model(
    max_length=training_cfg.get("generation_max_length"),
    num_beams=training_cfg.get("generation_num_beams"),
)
print("Eval metrics:", eval_metrics)

# 记录学习率与损失，可视化
log_history = pd.DataFrame(trainer.state.log_history)
log_history = log_history.dropna(subset=["loss"], how="all")
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
log_history.plot(x="step", y="loss", ax=axes[0], title="Training loss")
if "learning_rate" in log_history:
    log_history.plot(x="step", y="learning_rate", ax=axes[1], title="Learning rate")
plt.tight_layout()
plt.show()

## 6. 推理测试与权重保存
通过自定义 `generate_translation()` 对样例提示做推理，可选对比未 LoRA 的 baseline，并分别保存 LoRA 适配器与合并后的全量权重。

In [ ]:
COMPARE_BASELINE = False  # 若为 True，则额外载入未 LoRA 的 backbone 进行对比


def generate_translation(text: str, model_to_use=None, tokenizer_to_use=None, max_new_tokens: int = 128) -> str:
    model_to_use = model_to_use or trainer.model
    tokenizer_to_use = tokenizer_to_use or tokenizer
    prompt = data_cfg["prompt_template"].format(
        source_lang=data_cfg["source_language"],
        target_lang=data_cfg["target_language"],
        source_text=text,
    )
    inputs = tokenizer_to_use(prompt, return_tensors="pt").to(model_to_use.device)
    with torch.inference_mode():
        generated = model_to_use.generate(**inputs, max_new_tokens=max_new_tokens, num_beams=4)
    return tokenizer_to_use.decode(generated[0], skip_special_tokens=True)


demo_text = "The upcoming weekend weather looks great for hiking."
print("LoRA output:", generate_translation(demo_text))

if COMPARE_BASELINE:
    base_model = AutoModelForSeq2SeqLM.from_pretrained(
        model_cfg["name"],
        torch_dtype=dtype,
        device_map="auto" if torch.cuda.is_available() else None,
        cache_dir=project_cfg.get("cache_dir"),
    )
    base_model.eval()
    print("Baseline output:", generate_translation(demo_text, model_to_use=base_model))

# 保存 LoRA 适配器
lora_dir = output_dir / "lora_adapter"
lora_dir.mkdir(exist_ok=True)
trainer.model.save_pretrained(lora_dir)
tokenizer.save_pretrained(lora_dir)
print("LoRA adapter saved to", lora_dir)

# 合并权重并导出（用于纯推理部署）
merged_dir = output_dir / "merged_full_model"
merged_dir.mkdir(exist_ok=True)
merged_model = trainer.model.merge_and_unload()
merged_model.save_pretrained(merged_dir)
tokenizer.save_pretrained(merged_dir)
print("Merged model saved to", merged_dir)